In [32]:
import torch
import numpy as np

import matplotlib.pyplot as plt

import torch.optim as optim
import torch.nn as nn

In [33]:
# Inputs
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]

# Outputs
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

t_c = torch.tensor(  t_c   )
t_u = torch.tensor(  t_u   )

print(t_u)
print(t_c)

# t_u = 0.1 * t_u (for normalizing)

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000])
tensor([ 0.5000, 14.0000, 15.0000, 28.0000, 11.0000,  8.0000,  3.0000, -4.0000,
         6.0000, 13.0000, 21.0000])


## Splitting the Data

In [34]:
n_samples = t_u.shape[0]
n_samples

11

In [35]:
n_val = int(0.3 * n_samples)
n_val

3

In [36]:
shuffled_indeces = torch.randperm(n_samples)
shuffled_indeces

tensor([ 4,  7,  0, 10,  9,  5,  2,  8,  3,  1,  6])

In [37]:
train_indeces = shuffled_indeces[:-n_val]
val_indeces = shuffled_indeces[-n_val:]

print(train_indeces)

tensor([ 4,  7,  0, 10,  9,  5,  2,  8])


In [38]:
print(val_indeces)

tensor([3, 1, 6])


In [57]:
train_t_u = t_u[train_indeces]
train_t_c = t_c[train_indeces]

In [69]:
val_t_u = t_u[train_indeces]
val_t_c = t_c[train_indeces]

#### Note - There is always two equations happening - Inference Function and Loss Function for every model.

## Inference Function

In [70]:
def model(t_u, w, b):
    return w*t_u + b     # w & b are random values, e.g. 4 or -7, y(t_c) = w(t_u) * x + b

## Loss Function

In [71]:
def loss_fn(y_real, y_pred):
    squared_diffs = (y_real - y_pred) ** 2
    return squared_diffs.mean()

## Train Function

In [72]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    
    for epoch in range(1, n_epochs):
        
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        with torch.no_grad():
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        ## if epoch % 500 == 0:
        ##    print(train_loss.item())
        ##    print(val_loss.item())
        
            
    return params

## Function Calls

In [78]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
learning_rate = 0.0001
optimizer = optim.Adam( [params], lr = learning_rate )

## optimizer = optim.Adam( [params, lr = learning_rate] )

## Call Training Loop

In [79]:
result = training_loop(
    
    n_epochs = 3000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_u,
    val_t_u = val_t_u,
    train_t_c = train_t_c,
    val_t_c = val_t_c
)

print(result)

tensor([ 0.7222, -0.2786], requires_grad=True)


In [80]:
## Celsius from Farenheit
## 100 c = 212 f

my_Temp = model(32, * result)

print(my_Temp)

tensor(22.8331, grad_fn=<AddBackward0>)
